In [1]:
import utils

f0_predictor = utils.get_f0_predictor('rmvpe',
                                        sampling_rate=44100,
                                        hop_length=512,
                                        device='cuda',
                                        threshold=0.05)

In [2]:
import librosa
wav, sr = librosa.load("notebook/test_audios/ref_audio/24k_xigua.wav", sr=44100)

In [3]:
import time


f0, uv = f0_predictor.compute_f0_uv(wav)


In [8]:
import yaml
config = yaml.safe_load(open('configs/pretrained/config.yaml'))

In [11]:
config.update()

In [2]:
all_data = []
with open("Data/backup/raw_text_en.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())
with open("Data/raw_text.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())

In [4]:
import random

random.shuffle(all_data)

valid = all_data[:10]
train = all_data[10:]

In [6]:
with open("Data/sovits_svc/train.list",'w') as f:
    for line in train:
        line = line.strip()
        f.write(line + '\n')

with open("Data/sovits_svc/dev.list",'w') as f:
    for line in valid:
        line = line.strip()
        f.write(line + '\n')

In [3]:
with open("Data/raw_text.list",'w') as f:
    for line in all_data:
        line = line.strip()
        f.write(line + '\n')

In [13]:
import os
from pathlib import Path


dir = "ckpts/sunyanzi/audio_slice/sunyanzi"
p = Path(dir)

In [9]:
def move_files_simple(source_dir, target_dir):
    source_path = Path(source_dir)
    target_path = Path(target_dir)
    
    # 确保目标目录存在
    target_path.mkdir(parents=True, exist_ok=True)
    
    # 移动所有文件
    for file_path in source_path.glob("*.wav"):
        if file_path.is_file():
            file_path.rename(target_path / file_path.name)
            print(f"Moved: {file_path.name}")

In [ ]:
move_files_simple(p, p.parent.parent/'sunyanzi')

In [5]:
from pathlib import Path

t = Path("ckpts/sunyanzi/audio_slice/sunyanzi/sunyanzi_000000.wav")

In [6]:
"_".join(t.stem.split('_')[:2]) + '_f0.npy'

'sunyanzi_000000_f0.npy'

In [9]:
any(t.glob("g_*.pth"))

False

In [8]:
 list(t.glob("g_*.pth"))

[]

In [ ]:
import sys
import os
import torch
import torch.backends.cudnn as cudnn

def check_environment():
    print("=== Python Environment ===")
    print(f"Python executable: {sys.executable}")
    print(f"Python version: {sys.version}")
    
    print("\n=== PyTorch Information ===")
    print(f"PyTorch location: {torch.__file__}")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"CUDNN version: {cudnn.version()}")
    
    print("\n=== Environment Variables ===")
    print(f"CONDA_PREFIX: {os.environ.get('CONDA_PREFIX', 'Not set')}")
    print(f"LD_LIBRARY_PATH: {os.environ.get('LD_LIBRARY_PATH', 'Not set')}")
    
    if torch.cuda.is_available():
        print("\n=== CUDA Device Information ===")
        print(f"Current device: {torch.cuda.current_device()}")
        print(f"Device name: {torch.cuda.get_device_name()}")
        try:
            print(f"Device capability: {torch.cuda.get_device_capability()}")
        except AttributeError:
            pass
    
    print("\n=== Python Path ===")
    for path in sys.path:
        print(path)
    
    # 尝试定位 CUDA 相关库
    print("\n=== CUDA Library Check ===")
    torch_path = os.path.dirname(torch.__file__)
    print(f"Looking for CUDA libraries in: {torch_path}")
    
    # 检查常见的库文件位置
    cuda_paths = [
        os.path.join(torch_path, 'lib'),
        os.environ.get('CONDA_PREFIX', ''),
        '/usr/local/cuda/lib64',
        '/usr/lib/x86_64-linux-gnu'
    ]
    
    for path in cuda_paths:
        if os.path.exists(path):
            print(f"\nChecking {path}:")
            cuda_libs = [f for f in os.listdir(path) if 'cuda' in f.lower() or 'cudnn' in f.lower()]
            for lib in cuda_libs:
                print(f"Found: {lib}")

if __name__ == "__main__":
    check_environment()

In [2]:
import torch

In [5]:
# sovits底膜专用
def remove_optimizer(path):
    ckpt = torch.load(path, map_location='cpu')
    ckpt['optimizer'] = None
    torch.save(ckpt, path)
    

In [1]:
from pathlib import Path

model_dir = Path("ckpts/sunyanzi/diffusion")

In [5]:
f_list = list(model_dir.glob("*.pt"))

f_list.sort(
    key=lambda f: int("".join(filter(str.isdigit, f))))

In [7]:
len(f_list)

0